In [1]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import os
import numpy as np
from datetime import datetime
import dateutil.relativedelta
import configparser

In [3]:
%%capture
%pip install -r requirements.txt
!python -m nltk.downloader stopwords
!python -m nltk.downloader wordnet
!python -m nltk.downloader omw-1.4

In [2]:
last_updated_time = str(datetime.now() + dateutil.relativedelta.relativedelta(months=-1))
CurrentYear,CurrentMonth,CurrentDate = [int(i) for i in last_updated_time.split(" ")[0].split("-")]

In [59]:
if not os.path.exists('Last_Updated.txt'):
    last_updated_time = str(
        datetime.now() + dateutil.relativedelta.relativedelta(months=-1))
    CurrentYear,CurrentMonth,CurrentDate = [int(i) for i in last_updated_time.split(" ")[0].split("-")]
else:
    with open('Last_Updated.txt', 'r') as f:
        last_updated_time = f.read()
        CurrentYear,CurrentMonth,CurrentDate = [int(i) for i in last_updated_time.split(" ")[0].split("-")]
CurrentTime = datetime(CurrentYear,CurrentMonth,CurrentDate)

In [60]:
config = configparser.ConfigParser()
config.read_file(open('config.cfg'))
binance_coins= config.get('Telegram', 'Coins').replace("'","").strip('][').split(', ')
groups = config.get('Telegram', 'Groups').replace("'","").strip("][").split(', ')

In [42]:
print(groups)

['bestcryptotutor', 'binancefuturesignal', 'BitmexCourses', 'btctradingclub', 'coin_signals', 'CryptoClassicsVIP', 'CRYPTONOTESMOHSIN', 'cryptopapasignals', 'CryptoProfitCoach', 'cryptosignalalert', 'CryptoSignalCrazy', 'Dash2Trade', 'FedRussianInsiders', 'freebitmexsignals', 'godx_signals', 'List.txt', 'QualitySignalsChannel', 'The_CryptoLogy_Signals', 'UltimateCryptoSignals']


In [61]:
import snscrape.modules.telegram as telegram
import pandas as pd

tweets_list = []
for i in groups:
    print(i)
    for x,tel in enumerate(telegram.TelegramChannelScraper(str(i)).get_items()):
        x,y,z= [int(i) for i in str(tel.date).split(' ')[0].split('-')]
        Hello = datetime(x,y,z)
        if  (CurrentTime<Hello):
            break
        tweets_list.append([i,str(tel.date).split(' ')[0],str(tel.date).split(' ')[1].split('+')[0], tel.content])
    

bestcryptotutor
binancefuturesignal
BitmexCourses
btctradingclub
coin_signals
CryptoClassicsVIP
CRYPTONOTESMOHSIN
cryptopapasignals
CryptoProfitCoach
cryptosignalalert
CryptoSignalCrazy
Dash2Trade


No public post list for this user


FedRussianInsiders
freebitmexsignals
godx_signals
List.txt


No public post list for this user


QualitySignalsChannel
The_CryptoLogy_Signals
UltimateCryptoSignals


In [62]:
tel_df = pd.DataFrame(tweets_list, columns=['Name','Date','Time','Text'])


In [63]:
tel_df= tel_df.iloc[::-1]
tel_df

,Name,Date,Time,Text
2813,godx_signals,2022-08-06,13:28:16,Channel created
2812,godx_signals,2022-08-06,13:45:28,Lessons how to use our signals.Soon….
2811,godx_signals,2022-08-06,14:01:39,None
2810,godx_signals,2022-10-11,06:45:25,UNFI/USDT📈 BUY/LONGENTRY: 5.875-5.887💎TP1: 5.9...
2809,godx_signals,2022-10-11,15:47:12,UNFI/USDT📈 BUY/LONG ENTRY: 5.875-5.887 💎TP1:...
...,...,...,...,...
4,CryptoClassicsVIP,2022-11-08,19:32:16,#BTCUSDT Trade Type: long position Entry zon...
3,CryptoClassicsVIP,2022-11-09,21:22:57,#ALGOUSDT Trade Type: long position Entry zo...
2,CryptoClassicsVIP,2022-11-09,21:26:58,#BNBUSDT Trade Type: long position Entry zon...
1,CryptoClassicsVIP,2022-11-24,21:27:22,#LTCUSDT Trade Type: long position Entry zon...


In [ ]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
exclude.remove('.')
exclude.remove('-')
lemma = WordNetLemmatizer()
def text_cleaning(text):
    stop_free = ' '.join([word for word in str(text).split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

tel_df["Cleaned Text"]= tel_df["Text"].apply(text_cleaning)

In [ ]:
tel_df.groupby('Name').count()

In [ ]:
if not os.path.exists('Channels'):
    os.mkdir('Channels')
os.chdir(os.path.join(os.getcwd(),'Channels'))
for i in groups:
    if not os.path.exists(i):
        os.mkdir(i)
    os.chdir(os.path.join(os.getcwd(),i))
    tel_df[tel_df['Name']==i].to_csv('0_'+i+'.csv', mode='w', index=False, header=False)
    os.chdir('./Channels')
os.chdir("C:Users/Mohib/Desktop/FinalYear/Telegram")
with open('datetime.txt', 'w') as f:
    f.write(str(datetime.now()))

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
count = []
names = []
for group in groups:
    names.append(group)
    count.append(len(tel_df[tel_df['Name']==group]))
plt.bar(names,count)
plt.xticks(rotation=90)
plt.show()

Before Filter 1

In [ ]:
def extractor(i):
    i = str(i)
    suffixes = 'usdt|busd|futures'
    coinsregex = '|'.join(binance_coins)
    found = False
    text = re.sub(r'[^\w\s]', ' ', i)
    for s in text.split():
        if re.search("({})({})+|({})+({})|^({})$|[0-9]+({})|({})[0-9]+".format(coinsregex,suffixes,suffixes,coinsregex,coinsregex,coinsregex,coinsregex), s,re.IGNORECASE):
            found = True
            break
    if(found):
        return i
    else: return np.nan
tel_df['Cleaned Text'] = tel_df['Cleaned Text'].apply(extractor)
tel_df_fil1 = tel_df[tel_df['Cleaned Text'].notna()] 

In [ ]:
tel_df_fil1

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
count = list()
names = list()
for group in groups:
    names.append(group)
    count.append(len(tel_df_fil1[tel_df_fil1['Name']==group]))
plt.bar(names,count)
plt.xticks(rotation=90)
plt.show()

After Filter 1 

In [ ]:
os.chdir("/workspace/FinalYear/")
if not os.path.exists('Channels'):
    os.mkdir('Channels')
os.chdir(os.path.join(os.getcwd(),'Channels'))
for i in groups:
    if not os.path.exists(i):
        os.mkdir(i)
    os.chdir(os.path.join(os.getcwd(),i))
    tel_df_fil1[tel_df_fil1['Name']==i].to_csv('1_'+i+'.csv', mode='a', index=False, header=False)
    os.chdir('/workspace/FinalYear/Channels')
os.chdir("/workspace/FinalYear/")
with open('datetime.txt', 'w') as f:
    f.write(str(datetime.now()))